In [1]:
import pandas as pd

# Carga el archivo CSV con '|' como delimitador
df = pd.read_csv('r.csv', sep='|')

# Muestra las primeras filas del DataFrame
df.head()



,Id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,Num_reviews,Fecha_modificacion,Ingredientes
0,74138,Recetas de Aperitivos y tapas,Receta de Baleadas hondureñas,NaN,baja,2.0,10m,NaN,https://www.recetasgratis.net/receta-de-balead...,0,0,2020-04-06,"4 trotillas de harina,2 tazas de frijoles negr..."
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,NaN,baja,4.0,30m,Plato principal,https://www.recetasgratis.net/receta-de-tacos-...,0,0,2020-03-27,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,0,0,2020-03-25,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,0,0,2020-03-24,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,NaN,baja,3.0,30m,Entrante,https://www.recetasgratis.net/receta-de-masa-p...,0,0,2020-03-03,"3 tazas de harina de trigo (420 gramos),1 pizc..."


In [2]:
df = df.drop("Valoracion", axis=1)
df = df.drop("Num_comentarios", axis=1)
df = df.drop("Num_reviews", axis=1)
df = df.drop("Fecha_modificacion", axis=1)
df = df.drop("Tipo", axis=1)

In [3]:
df.head()

,Id,Categoria,Nombre,Dificultad,Num_comensales,Tiempo,Link_receta,Ingredientes
0,74138,Recetas de Aperitivos y tapas,Receta de Baleadas hondureñas,baja,2.0,10m,https://www.recetasgratis.net/receta-de-balead...,"4 trotillas de harina,2 tazas de frijoles negr..."
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,baja,4.0,30m,https://www.recetasgratis.net/receta-de-tacos-...,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,muy baja,5.0,30m,https://www.recetasgratis.net/receta-de-burrit...,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,baja,4.0,45m,https://www.recetasgratis.net/receta-de-croque...,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,baja,3.0,30m,https://www.recetasgratis.net/receta-de-masa-p...,"3 tazas de harina de trigo (420 gramos),1 pizc..."


In [4]:
df = df.dropna(subset=["Ingredientes"])

In [5]:
df.isnull().sum()

Id                   0
Categoria            0
Nombre               0
Dificultad        7386
Num_comensales    9305
Tiempo            9307
Link_receta          0
Ingredientes         0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22150 entries, 0 to 22373
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              22150 non-null  int64  
 1   Categoria       22150 non-null  object 
 2   Nombre          22150 non-null  object 
 3   Dificultad      14764 non-null  object 
 4   Num_comensales  12845 non-null  float64
 5   Tiempo          12843 non-null  object 
 6   Link_receta     22150 non-null  object 
 7   Ingredientes    22150 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.5+ MB


In [7]:
df.shape

(22150, 8)

In [8]:
import pandas as pd
from antlr4 import *
from recipesLexer import recipesLexer
from recipesParser import recipesParser

In [9]:
# Definir una función para usar el parser de ANTLR4
def parsear_ingredientes(consulta):
    input_stream = InputStream(consulta)
    lexer = recipesLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = recipesParser(stream)
    tree = parser.consulta()
    
    ingredientes = []
    for child in tree.getChildren():
        if isinstance(child, recipesParser.IngredienteContext):
            ingredientes.append(child.getText().lower())
    
    return ingredientes

In [10]:
# Función para filtrar recetas basadas en los ingredientes ingresados por el usuario
def filtrar_recetas(consulta, df):
    ingredientes = parsear_ingredientes(consulta)
    recetas_filtradas = df[df['Ingredientes'].apply(lambda x: all(ing in x.lower() for ing in ingredientes))]
    return recetas_filtradas

# Ejemplo de consulta ingresada por el usuario
consulta_usuario = "tomate, queso, albahaca, pollo"

In [11]:
# Filtrar recetas
if 'Ingredientes' in df.columns:
    recetas = filtrar_recetas(consulta_usuario, df)
    # Ajustar 'title' según el nombre de la columna en tu DataFrame, por ejemplo 'recipe_name'
    print('encontro')
else:
    print("La columna 'all_ingredients' no existe en el DataFrame. Asegúrate de haberla creado correctamente.")

encontro


In [12]:
df_recetas = pd.DataFrame(recetas, columns=['Nombre'])


In [13]:
def buscar_recetas():
    consulta = "".join(selected_ingredients)
    print("consulta: "+consulta)
    if 'Ingredientes' in df.columns:
        recetas = search_recipes(consulta, df)
        # Limpiar el Treeview
        for row in tree.get_children():
            tree.delete(row)
        if not recetas.empty:
            categorias = recetas['Categoria'].unique()
            for categoria in categorias:
                parent = tree.insert("", tk.END, values=(categoria, "", ""), open=True)
                recetas_categoria = recetas[recetas['Categoria'] == categoria]
                for index, row in recetas_categoria.iterrows():
                    tree.insert(parent, tk.END, values=("", row['Nombre'], row['Link_receta']))
            consulta = ""
        else:
            messagebox.showinfo("Sin resultados", "No se encontraron recetas con los ingredientes seleccionados.")
            consulta = ""
    else:
        messagebox.showerror("Error", "La columna 'Ingredientes' no existe en el DataFrame.")
        consulta = ""


In [14]:
import pandas as pd
from antlr4 import *
from recipesLexer import recipesLexer
from recipesParser import recipesParser
from recipesListener import recipesListener
import re
# Función para analizar la consulta usando ANTLR
def parse_consulta(consulta):
    input_stream = InputStream(consulta)
    lexer = recipesLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = recipesParser(stream)
    tree = parser.consulta()
    listener = RecipeListener()
    walker = ParseTreeWalker()
    walker.walk(listener, tree)
    return listener

# Listener para extraer los ingredientes de la consulta
class RecipeListener(recipesListener):
    def __init__(self):
        self.ingredientes = []
        self.logical_ops = []
        self.current_ingrediente = []

    def exitIngrediente(self, ctx: recipesParser.IngredienteContext):
        Ingrediente = ctx.getText()
        self.current_ingrediente.append(Ingrediente)

    def exitCondicion(self, ctx: recipesParser.CondicionContext):
        if self.current_ingrediente:
            self.ingredientes.append(" ".join(self.current_ingrediente))
            self.current_ingrediente = []

    def exitConsulta(self, ctx: recipesParser.ConsultaContext):
        for child in ctx.getChildren():
            if isinstance(child, TerminalNode) and child.getText() in [",", "|"]:
                self.logical_ops.append(child.getText())

def search_recipes(consulta, df):
    listener = parse_consulta(consulta)
    print(listener)
    ingredientes = listener.ingredientes
    logical_ops = listener.logical_ops
    print(ingredientes)
    def contains_ingrediente(ingrediente, recipe_ingredientes):
        # Agregar un espacio entre el número y la palabra si el número está seguido de letras
        ingrediente_pattern = re.sub(r'(\d+)([a-zA-Z])', r'\1 \2', ingrediente.lower())
        return bool(re.search(r'\b' + re.escape(ingrediente_pattern) + r'\b', recipe_ingredientes.lower()))

    # Inicialmente, seleccionamos todas las recetas
    filtered_df = df.copy()

    if not logical_ops:
        for ingrediente in ingredientes:
            filtered_df = filtered_df[filtered_df['Ingredientes'].apply(lambda x: contains_ingrediente(ingrediente, x))]
    else:
        result_mask = None
        for i, ingrediente in enumerate(ingredientes):
            ingrediente_mask = df['Ingredientes'].apply(lambda x: contains_ingrediente(ingrediente, x))
            if result_mask is None:
                result_mask = ingrediente_mask
            else:
                if logical_ops[i - 1] == ',':
                    result_mask = result_mask & ingrediente_mask
                elif logical_ops[i - 1] == '|':
                    result_mask = result_mask | ingrediente_mask

        filtered_df = df[result_mask]

    return filtered_df


In [15]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import scrolledtext
import webbrowser

In [16]:
# Función para agregar ingredientes y desactivar botones
def add_ingredient(ingredient):
    global selected_ingredients
    if ingredient in [",", "|", "(", ")"]:
        selected_ingredients += ingredient  # Añadir directamente el separador
    else:
        if ingredient not in selected_ingredients:
            selected_ingredients.append(ingredient)
    update_ingredients_label()

In [17]:
def convert_logical_operators(consulta):
    consulta = consulta.replace('AND', ',')
    consulta = consulta.replace('OR', '|')
    return consulta

In [18]:
def update_ingredients_label():
    ingredients_label.config(text="Ingredientes seleccionados: " + "".join(selected_ingredients))

In [19]:
def on_treeview_click(event):
    item = tree.selection()
    if item:
        item = item[0]
        link = tree.item(item, "values")[2]
        webbrowser.open(link)

In [20]:
# Crear ventana principal
window = tk.Tk()
window.title("Aplicación de Recetas")

''

In [21]:
# Lista para almacenar los ingredientes seleccionados
selected_ingredients = []

In [22]:
# Lista de ingredientes disponibles
available_ingredients = ["Pollo", "Zanahoria", "Papa", "Tomate", "Cebolla","carne", ",", "|", "(", ")",
                        "1 ","2 ","3 ","4 ","5 ","6 ","7 ","8 ","9 ","0 "]

In [23]:
# Crear y posicionar los widgets de manera dinámica
for idx, ingredient in enumerate(available_ingredients):
    button = tk.Button(window, text=ingredient, command=lambda ingr=ingredient: add_ingredient(ingr))
    button.grid(row=0, column=idx, padx=5, pady=5, sticky="ew")

In [24]:
ingredients_label = tk.Label(window, text="Ingredientes seleccionados: ")

button_buscar = tk.Button(window, text="Buscar", command=buscar_recetas)

result_label = tk.Label(window, text="Resultados:")

In [25]:
# Crear Treeview para mostrar los resultados
tree = ttk.Treeview(window, columns=("Categoria", "Nombre", "Link_receta"), show='headings')
tree.heading("Categoria", text="Categoria")
tree.heading("Nombre", text="Nombre")
tree.heading("Link_receta", text="Link_receta")
tree.column("Categoria", width=100)
tree.column("Nombre", width=200)
tree.column("Link_receta", width=200)

{}

In [26]:
# Añadir evento de clic al Treeview
tree.bind("<Double-1>", on_treeview_click)

'2547544692288on_treeview_click'

In [27]:
# Posicionar widgets en la ventana usando grid
ingredients_label.grid(row=1, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
button_buscar.grid(row=2, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
result_label.grid(row=3, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")
tree.grid(row=4, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="nsew")

In [28]:
# Configurar que las columnas y filas se expandan
for i in range(len(available_ingredients)):
    window.grid_columnconfigure(i, weight=1)
window.grid_rowconfigure(4, weight=1)

In [29]:
# Iniciar el bucle principal de la ventana
window.mainloop()

In [43]:
import tkinter as tk
from tkinter import ttk, messagebox
import webbrowser
from PIL import Image, ImageDraw, ImageFont, ImageTk
import time

def add_ingredient(ingredient):
    global selected_ingredients
    if ingredient in [",", "|", "(", ")"]:
        selected_ingredients += ingredient  # Añadir directamente el separador
        #selected_ingredients.append(ingredient)
        update_ingredient_buttons()
    else:
        if ingredient not in selected_ingredients:
            selected_ingredients.append(ingredient)
            if ingredient in available_ingredients:
                available_ingredients.remove(ingredient)
            update_ingredient_buttons()
    update_ingredients_label()


def remove_ingredient(ingredient):
    global selected_ingredients, available_ingredients
    
    # Buscar la última ocurrencia del ingrediente u operador y eliminarlo
    if ingredient in selected_ingredients[::-1]:  # Buscar desde el final hacia el principio
        index = len(selected_ingredients) - 1 - selected_ingredients[::-1].index(ingredient)
        removed_ingredient = selected_ingredients.pop(index)  # Eliminar el elemento encontrado
        
        # Agregar el ingrediente de vuelta a available_ingredients
        if removed_ingredient not in available_ingredients:
            available_ingredients.append(removed_ingredient)
        
    update_ingredient_buttons()
    update_ingredients_label()
    ordenar()
        
def ordenar():
    # Eliminar duplicados y ordenar la lista
    unique_ingredients = list(set(available_ingredients))
    unique_ingredients.sort()
    
    # Actualizar la lista original con los ingredientes únicos y ordenados
    available_ingredients[:] = unique_ingredients
    
    # Llamar a las funciones para actualizar la interfaz de usuario
    update_ingredient_buttons()
    update_ingredients_label()

def update_ingredients_label():
    ingredients_label.config(text="Ingredientes seleccionados: " + "".join(selected_ingredients))

def on_treeview_click(event):
    item = tree.selection()
    if item:
        item = item[0]
        values = tree.item(item, "values")
        # Solo abrir el enlace si es una receta (tiene un enlace en values[2])
        if values[2]:
            link = values[2]
            webbrowser.open(link)

def create_gradient_image(width, height, color1, color2):
    image = Image.new("RGBA", (width, height), color1 + (0,))
    draw = ImageDraw.Draw(image)
    for i in range(height):
        ratio = i / height
        r = int(color1[0] * (1 - ratio) + color2[0] * ratio)
        g = int(color1[1] * (1 - ratio) + color2[1] * ratio)
        b = int(color1[2] * (1 - ratio) + color2[2] * ratio)
        a = int(255 * ratio)
        draw.line((0, i, width, i), fill=(r, g, b, a))
    return image

def create_rounded_gradient_button(width, height, color1, color2, radius, text, text_color="white"):
    gradient_image = create_gradient_image(width, height, color1, color2)
    mask = Image.new("L", (width, height), 0)
    draw = ImageDraw.Draw(mask)
    draw.rounded_rectangle((0, 0, width, height), radius, fill=255)
    gradient_image.putalpha(mask)

    draw = ImageDraw.Draw(gradient_image)
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()

    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    text_x = (width - text_width) // 2
    text_y = (height - text_height) // 2
    draw.text((text_x, text_y), text, font=font, fill=text_color)

    return ImageTk.PhotoImage(gradient_image)

def update_ingredient_buttons():
    for widget in ingredients_frame.winfo_children():
        widget.destroy()
    for widget in selected_ingredients_frame.winfo_children():
        widget.destroy()

    def place_buttons(frame, ingredients, frame_dict, button_func, color1, color2, button_size=(120, 40), remove_button=False):
        row, col = 0, 0
        max_columns = max(frame.winfo_width() // (button_size[0] + 10), 1)

        for ingredient in ingredients:
            if col >= max_columns:
                row += 1
                col = 0

            button_image = create_rounded_gradient_button(button_size[0], button_size[1], color1, color2, 20, ingredient)
            button = tk.Button(frame, image=button_image, command=lambda ingr=ingredient: button_func(ingr), compound="center", bd=0, bg='#FFFAF0')
            button.image = button_image  # Keep a reference to avoid garbage collection
            button.grid(row=row, column=col, padx=5, pady=5, sticky="ew")

            if remove_button:
                x_button_image = create_rounded_gradient_button(30, 30, (220, 20, 60), (255, 69, 0), 15, "X")
                x_button = tk.Button(frame, image=x_button_image, command=lambda ingr=ingredient: remove_ingredient(ingr), compound="center", bd=0, bg='#FFFAF0')
                x_button.image = x_button_image  # Keep a reference to avoid garbage collection
                x_button.grid(row=row, column=col + 1, padx=(0, 5), pady=5, sticky="w")

            col += 2

    place_buttons(ingredients_frame, available_ingredients, ingredient_buttons, add_ingredient, (255, 99, 71), (255, 165, 0))
    place_buttons(selected_ingredients_frame, selected_ingredients, selected_ingredient_buttons, remove_ingredient, (255, 69, 0), (255, 140, 0), remove_button=True)

# Crear ventana principal
window = tk.Tk()
window.title("Aplicación de Recetas")
window.geometry("800x600")

window.configure(bg='#FFFAF0')

bg_image = Image.open("1.jpg")
bg_image = bg_image.resize((800, 600), Image.LANCZOS)
bg_photo = ImageTk.PhotoImage(bg_image)

def draw_background(canvas, image):
    canvas.delete("all")
    canvas_width = canvas.winfo_width()
    canvas_height = canvas.winfo_height()
    img_width = image.width()
    img_height = image.height()

    for x in range(0, canvas_width, img_width):
        for y in range(0, canvas_height, img_height):
            canvas.create_image(x, y, image=image, anchor='nw')

canvas = tk.Canvas(window, width=800, height=600)
canvas.grid(row=0, column=0, rowspan=6, columnspan=len(available_ingredients), sticky="nsew")
canvas.create_image(0, 0, image=bg_photo, anchor='nw')

selected_ingredients = []
available_ingredients = ["Pollo", "Zanahoria", "Papa", "Tomate", "Cebolla","carne", ",", "|", "(", ")",
                        "1 ","2 ","3 ","4 ","5 ","6 ","7 ","8 ","9 ","0 "]

ingredient_buttons = {}
selected_ingredient_buttons = {}

ingredients_frame = tk.Frame(window, bg='#FFFAF0')
ingredients_frame.grid(row=1, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")

selected_ingredients_frame = tk.Frame(window, bg='#FFFAF0')
selected_ingredients_frame.grid(row=2, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")

ingredients_label = tk.Label(window, text="Ingredientes seleccionados: ", bg='#FFFAF0', font=('Helvetica', 16, 'bold', 'italic'))
ingredients_label.grid(row=0, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")

search_button_frame = tk.Frame(window, bg='#FFFAF0')
search_button_frame.grid(row=3, column=0, columnspan=len(available_ingredients), pady=10)

button_image = create_rounded_gradient_button(120, 40, (50, 205, 50), (34, 139, 34), 20, "Buscar")
button_buscar = tk.Button(search_button_frame, image=button_image, command=buscar_recetas, compound="center", bd=0, bg='#FFFAF0')
button_buscar.image = button_image  # Keep a reference to avoid garbage collection
button_buscar.pack()

result_label = tk.Label(window, text="Resultados:", bg='#FFFAF0', font=('Helvetica', 16, 'bold', 'italic'))
result_label.grid(row=4, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="ew")

tree_frame = tk.Frame(window, bg='#FFFAF0')
tree_frame.grid(row=5, column=0, columnspan=len(available_ingredients), padx=5, pady=5, sticky="nsew")

tree = ttk.Treeview(tree_frame, columns=("Categoria", "Nombre", "Link_receta"), show='headings', style="mystyle.Treeview")
tree.heading("Categoria", text="Categoria")
tree.heading("Nombre", text="Nombre")
tree.heading("Link_receta", text="Link_receta")
tree.column("Categoria", width=150)
tree.column("Nombre", width=200)
tree.column("Link_receta", width=300)

scrollbar = ttk.Scrollbar(tree_frame, orient="vertical", command=tree.yview)
tree.configure(yscrollcommand=scrollbar.set)
scrollbar.pack(side="right", fill="y")

tree.pack(expand=True, fill="both")
tree.bind("<Double-1>", on_treeview_click)

for i in range(len(available_ingredients)):
    window.grid_columnconfigure(i, weight=1)
window.grid_rowconfigure(1, weight=1)
window.grid_rowconfigure(2, weight=1)
window.grid_rowconfigure(3, weight=1)
window.grid_rowconfigure(4, weight=1)
window.grid_rowconfigure(5, weight=10)

style = ttk.Style()
style.configure("mystyle.Treeview", highlightthickness=0, bd=0, font=('Helvetica', 10))
style.configure("mystyle.Treeview.Heading", font=('Helvetica', 12, 'bold'))
style.map("mystyle.Treeview", background=[('selected', '#32CD32')])

def on_resize(event):
    draw_background(canvas, bg_photo)
    update_ingredient_buttons()

canvas.bind("<Configure>", on_resize)

window.mainloop()

consulta: Pollo,Cebolla,Papa
['Pollo', 'Cebolla', 'Papa']
consulta: (Papa|Cebolla),(Pollo|carne)
['Papa', 'Cebolla', 'Pollo', 'carne']
consulta: (Papa|Cebolla),(Pollo|carne),Tomate,Zanahoria
['Papa', 'Cebolla', 'Pollo', 'carne', 'Tomate', 'Zanahoria']
consulta: (Papa|Cebolla)
['Papa', 'Cebolla']
